## Prompting gpt-3.5-turbo to get N adversarial negative examples for each NE in pileNER

In [1]:
import json
import sys
import os

from dotenv import load_dotenv
from openai import OpenAI
import tiktoken 

# my libraries
sys.path.append("/Users/andrew/ExpertAI/MSEQA_for_NER/src/MSEQA_4_NER")
from data_handlers import data_handler_pileNER

In [2]:
# Load API key from .env
load_dotenv('./.env')
print(os.environ.get('OPENAI_API_KEY') is not None)

True


In [3]:
client = OpenAI()

In [4]:
# gpt_model = "gpt-3.5-turbo-1106"
gpt_model = "gpt-3.5-turbo-0125" 

In [5]:
dataset_name = "pileNER"
with open(f"../data_handlers/questions/{dataset_name}/all_423_NE_definitions.json", 'r') as file:
        guidelines_per_ne_type = json.load(file)
ne_types_list = list(guidelines_per_ne_type.keys())

print("NE types:")
print(len(ne_types_list))
print(ne_types_list)

NE types:
423
['person', 'organization', 'location', 'concept', 'product', 'variable', 'date', 'medical condition', 'object', 'technology', 'chemical', 'software', 'event', 'number', 'disease', 'attribute', 'protein', 'group', 'material', 'measurement', 'function', 'nationality', 'country', 'class', 'process', 'title', 'animal', 'component', 'condition', 'substance', 'food', 'city', 'activity', 'type', 'company', 'time', 'method', 'property', 'organism', 'drug', 'medical procedure', 'treatment', 'profession', 'cell type', 'anatomical structure', 'job title', 'data', 'quantity', 'sports team', 'biological process', 'data type', 'programming language', 'occupation', 'file', 'body part', 'medical treatment', 'language', 'chemical compound', 'gene', 'state', 'law', 'action', 'website', 'library', 'facility', 'publication', 'tool', 'field of study', 'compound', 'document', 'organ', 'abbreviation', 'character', 'brand', 'device', 'operating system', 'service', 'technique', 'species', 'sympto

In [6]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-1106"):
    """Return the number of tokens used in the input prompt."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        # print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [7]:
def fix_guidelines(gpt_definition):
    if not gpt_definition.endswith("}"):
        if not gpt_definition.endswith("\""):
            gpt_definition += "\""
        gpt_definition += "}"
    return gpt_definition

In [8]:
print(guidelines_per_ne_type['measurement']['gpt_answer'])

gpt_definition_fixed = fix_guidelines(guidelines_per_ne_type['measurement']['gpt_answer'])

print(gpt_definition_fixed)

{"Definition": "'measurement' refers to numerical or quantifiable entities such as units of length, mass, time, or data metrics used in scientific, engineering, or quantitative contexts.", "Guidelines": "Avoid labeling abstract concepts that are not quantifiable, such as 'happiness' or 'risk'. Exercise caution with ambiguous terms like 'feet' (could refer to the body part or a unit of measurement). Be mindful of context to distinguish between literal and figurative use of measurements, e
{"Definition": "'measurement' refers to numerical or quantifiable entities such as units of length, mass, time, or data metrics used in scientific, engineering, or quantitative contexts.", "Guidelines": "Avoid labeling abstract concepts that are not quantifiable, such as 'happiness' or 'risk'. Exercise caution with ambiguous terms like 'feet' (could refer to the body part or a unit of measurement). Be mindful of context to distinguish between literal and figurative use of measurements, e"}


# PROMPT formulation

In [12]:
n_sentences_to_generate = 3

ne_as_example = 'date'
ne_guidelines_example = fix_guidelines(guidelines_per_ne_type[ne_as_example]['gpt_answer'])
example_prompt = f"You will be given a Named Entity and its associated annotation guidelines (delimited by triple quotes). \nCarefully read the Definition and Guidelines before answering to the instruction in the end. \n\"\"\"\nNamed Entity: \'{ne_as_example}\', {ne_guidelines_example}\n\"\"\"" 
example_prompt += f"\nInstruction: Leverage on the provided Definition and annotation Guidelines to generate {n_sentences_to_generate} adversarial sentences containing negative occurrences of this Named Entity (negative occurrences are what guidelines say not to label). While doing so, be extremely careful NOT to include any positive occurrences of this Named Entity (what would instead be labeled as this Named Entity). For each sentence provide an explanation. Ensure to output {n_sentences_to_generate} adversarial sentences.\n"

print(example_prompt)

You will be given a Named Entity and its associated annotation guidelines (delimited by triple quotes). 
Carefully read the Definition and Guidelines before answering to the instruction in the end. 
"""
Named Entity: 'date', {"Definition": "'date' refers to specific points in time, including days, months, years, and relative time expressions like 'Week 2'.", "Guidelines": "Avoid labeling non-specific time references like 'recently' or 'soon'. Exercise caution with ambiguous terms like 'May' (month or verb) and 'Wednesday Adams' (person's name which includes a day of the week)."}
"""
Instruction: Leverage on the provided definition and annotation guidelines to generate 3 adversarial sentences containing negative occurrences of this Named Entity (negative occurrences are what guidelines say not to label). While doing so, be extremely careful NOT to include any positive occurrences of this Named Entity (what would instead be labeled as this Named Entity). For each sentence provide an expl

In [13]:
first_negative_example_answer = {"adversarial_sentence": "Jane recently completed her thesis on environmental sustainability, drawing from a plethora of cutting-edge research findings to present a comprehensive analysis of current challenges and potential solutions.", "explanation": "Non-specific time reference 'recently'"}
second_negative_example_answer = {"adversarial_sentence": "May I kindly request your assistance in resolving this matter as swiftly as possible?\"", "explanation": "Ambiguous term 'May' used here as verb and not as month."}
third_negative_example_answer = {"adversarial_sentence": "In the dimly lit auditorium, the audience couldn't help but be captivated by the eerie presence of Wednesday Addams as she delivered her lines with a chilling calmness during the school play, reminding everyone of her unforgettable character from the Addams Family series.",  "explanation": "Ambiguous term 'Wednesday' used here as person name and not as day of the week."} 
example_answer = json.dumps(f"[{first_negative_example_answer}, {second_negative_example_answer}, {third_negative_example_answer}]")
print(example_answer)
    

"[{'adversarial_sentence': 'Jane recently completed her thesis on environmental sustainability, drawing from a plethora of cutting-edge research findings to present a comprehensive analysis of current challenges and potential solutions.', 'explanation': \"Non-specific time reference 'recently'\"}, {'adversarial_sentence': 'May I kindly request your assistance in resolving this matter as swiftly as possible?\"', 'explanation': \"Ambiguous term 'May' used here as verb and not as month.\"}, {'adversarial_sentence': \"In the dimly lit auditorium, the audience couldn't help but be captivated by the eerie presence of Wednesday Addams as she delivered her lines with a chilling calmness during the school play, reminding everyone of her unforgettable character from the Addams Family series.\", 'explanation': \"Ambiguous term 'Wednesday' used here as person name and not as day of the week.\"}]"


In [14]:
def get_prompt_for_a_ne(named_entity, guidelines_per_ne_type, gpt_model_name, max_output_tokens, n_sentences_to_generate=3):
    # system message
    system_message = "You are a helpful NER data annotator designed to output JSON."
    # user prompt example on date
    ne_as_example = 'date'
    ne_guidelines_example = fix_guidelines(guidelines_per_ne_type[ne_as_example]['gpt_answer'])
    example_prompt = f"You will be given a Named Entity and its associated annotation guidelines (delimited by triple quotes). \nCarefully read the Definition and Guidelines before answering to the instruction in the end. \n\"\"\"\nNamed Entity: \'{ne_as_example}\', {ne_guidelines_example}\n\"\"\"" 
    example_prompt += f"\nInstruction: Leverage on the provided Definition and annotation Guidelines to generate {n_sentences_to_generate} adversarial sentences containing negative occurrences of this Named Entity (negative occurrences are what the guidelines say not to label). While doing so, be extremely careful NOT to include any positive occurrences of this Named Entity (what would instead be labeled as this Named Entity). For each sentence provide an explanation. Ensure to output {n_sentences_to_generate} adversarial sentences.\n"
    
    # Output a list of length {n_sentences_to_generate} where each item is a dict with fields \"adversarial_sentence\" and \"explanation\".

    # example answer
    first_negative_example_answer = {"adversarial_sentence": "Jane recently completed her thesis on environmental sustainability, drawing from a plethora of cutting-edge research findings to present a comprehensive analysis of current challenges and potential solutions.", "explanation": "Non-specific time reference 'recently'"}
    second_negative_example_answer = {"adversarial_sentence": "May I kindly request your assistance in resolving this matter as swiftly as possible?\"", "explanation": "Ambiguous term 'May' used here as verb and not as month."}
    third_negative_example_answer = {"adversarial_sentence": "In the dimly lit auditorium, the audience couldn't help but be captivated by the eerie presence of Wednesday Addams as she delivered her lines with a chilling calmness during the school play, reminding everyone of her unforgettable character from the Addams Family series.",  "explanation": "Ambiguous term 'Wednesday' used here as person name and not as day of the week."} 
    example_answer = json.dumps(f"[{first_negative_example_answer}, {second_negative_example_answer}, {third_negative_example_answer}]")
    #example_answer = json.dumps([first_negative_example_answer, second_negative_example_answer, third_negative_example_answer])
    
    # real prompt
    ne_guidelines = fix_guidelines(guidelines_per_ne_type[named_entity]['gpt_answer'])
    real_prompt = f"Now do the same for \"\"\" Named Entity: \'{named_entity}\', {ne_guidelines} \"\"\"\n"

    # message to send
    messages_to_send = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": example_prompt
    },
    {
        "role": "system",
        "content": example_answer
    },
    {
        "role": "user",
        "content": real_prompt
    }
    ]

    n_input_tokens = num_tokens_from_messages(messages_to_send, gpt_model)
    # print(messages_to_send)

    for message in messages_to_send:
        print(message)
    
    if n_input_tokens > 1000:
        raise ValueError(f"Too many input tokens in messages_to_send: {len(messages_to_send)}, {messages_to_send}")

    print(f"Sending prompt for NE: {named_entity}...")
    
    completion = client.chat.completions.create(
        messages=messages_to_send,
        model=gpt_model_name,
        max_tokens=max_output_tokens,
        response_format={ "type": "json_object" }, 
        temperature=0
    )

    choice = completion.choices[0]
    finish_reason = choice.finish_reason
    message = choice.message.content

    #if finish_reason == 'length' and message[-2:] != "\"}":
        #message += "\"}"

    return {"named_entity": named_entity,
            "prompt_length": completion.usage.prompt_tokens,
            "output_length": completion.usage.completion_tokens,
            "annotation_guidelines": json.loads(ne_guidelines),
            "negative_sentences": message,
            "finish_reason": finish_reason
        }
    

In [59]:
# NO FAKE SYSTEM MESSAGE BUT DIRECT PROMPT

def get_prompt_for_a_ne(named_entity, guidelines_per_ne_type, gpt_model_name, max_output_tokens, n_sentences_to_generate=3):
    # system message
    system_message = "You are a helpful NER data annotator designed to output JSON."
    # user prompt example on date
    ne_guidelines = fix_guidelines(guidelines_per_ne_type[named_entity]['gpt_answer'])
    example_prompt = f"You will be given a Named Entity and its associated annotation guidelines (delimited by triple quotes). \nCarefully read the Definition and Guidelines before answering to the instruction in the end. \n\"\"\"\nNamed Entity: \'{named_entity}\', {ne_guidelines}\n\"\"\"" 
    example_prompt += f"\nInstruction: Leverage on the provided Definition and annotation Guidelines to generate {n_sentences_to_generate} adversarial sentences containing negative occurrences of this Named Entity (negative occurrences are what the guidelines say not to label). While doing so, be extremely careful NOT to include any positive occurrences of this Named Entity (what would instead be labeled as this Named Entity). For each sentence provide a short explanation. Ensure to output {n_sentences_to_generate} adversarial sentences. The sentences should not be too short.\n"
    
    # Output a list of length {n_sentences_to_generate} where each item is a dict with fields \"adversarial_sentence\" and \"explanation\".

    # message to send
    messages_to_send = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": example_prompt
    }
    ]

    n_input_tokens = num_tokens_from_messages(messages_to_send, gpt_model)
    # print(messages_to_send)

    for message in messages_to_send:
        print(message)
    
    if n_input_tokens > 1000:
        raise ValueError(f"Too many input tokens in messages_to_send: {len(messages_to_send)}, {messages_to_send}")

    print(f"Sending prompt for NE: {named_entity}...")
    
    completion = client.chat.completions.create(
        messages=messages_to_send,
        model=gpt_model_name,
        max_tokens=max_output_tokens,
        response_format={ "type": "json_object" }, 
        temperature=0
    )

    choice = completion.choices[0]
    finish_reason = choice.finish_reason
    message = choice.message.content

    #if finish_reason == 'length' and message[-2:] != "\"}":
        #message += "\"}"

    return {"named_entity": named_entity,
            "prompt_length": completion.usage.prompt_tokens,
            "output_length": completion.usage.completion_tokens,
            "annotation_guidelines": json.loads(ne_guidelines),
            "negative_sentences": message,
            "finish_reason": finish_reason
        }
    

In [57]:
response = get_prompt_for_a_ne('framework', guidelines_per_ne_type, gpt_model, max_output_tokens=1000, n_sentences_to_generate=3)

{'role': 'system', 'content': 'You are a helpful NER data annotator designed to output JSON.'}
{'role': 'user', 'content': 'You will be given a Named Entity and its associated annotation guidelines (delimited by triple quotes). \nCarefully read the Definition and Guidelines before answering to the instruction in the end. \n"""\nNamed Entity: \'framework\', {"Definition": "\'framework\' in the context of NER refers to software development platforms, libraries, or tools used for building applications.", "Guidelines": "Avoid labeling general technology terms (e.g., \'library\', \'tool\') unless specifically referring to a named software framework. Exercise caution when labeling ambiguous terms that may refer to multiple meanings, such as \'Java\' (programming language or island) or \'Swift\' (programming language or adjective)."}\n\n"""\nInstruction: Leverage on the provided Definition and annotation Guidelines to generate 3 adversarial sentences containing negative occurrences of this Na

In [55]:
print(response)

{'named_entity': 'framework', 'prompt_length': 251, 'output_length': 213, 'annotation_guidelines': {'Definition': "'framework' in the context of NER refers to software development platforms, libraries, or tools used for building applications.", 'Guidelines': "Avoid labeling general technology terms (e.g., 'library', 'tool') unless specifically referring to a named software framework. Exercise caution when labeling ambiguous terms that may refer to multiple meanings, such as 'Java' (programming language or island) or 'Swift' (programming language or adjective)."}, 'negative_sentences': '{\n  "adversarial_sentences": [\n    {\n      "sentence": "I need to update the Java framework on my computer.",\n      "explanation": "This sentence contains the term \'Java\' which is an ambiguous term that can refer to a programming language or an island. It should not be labeled as the Named Entity \'framework\' as per the guidelines."\n    },\n    {\n      "sentence": "The Swift framework of decisio

In [58]:
print(response['negative_sentences'])

{
  "adversarial_sentences": [
    {
      "sentence": "I need to sharpen my skills in using various libraries and tools for software development.",
      "explanation": "This sentence contains the negative occurrences of 'library' and 'tool' which are to be avoided as labels for the Named Entity 'framework'."
    },
    {
      "sentence": "The team decided to explore different platforms for building applications to enhance their productivity.",
      "explanation": "Here, 'platforms' is a negative occurrence as it refers to software development platforms, which should not be labeled as the Named Entity 'framework'."
    },
    {
      "sentence": "She swiftly completed the project using a combination of programming languages and tools.",
      "explanation": "The use of 'programming languages' and 'tools' in this sentence are negative occurrences that should not be labeled as the Named Entity 'framework'."
    }
  ]
}


In [16]:
print(response['negative_sentences'])

{
  "adversarial_sentences": [
    {
      "adversarial_sentence": "The team is planning to launch the new product soon, aiming to capture a larger market share before the end of the fiscal year.",
      "explanation": "Non-specific time reference 'soon'"
    },
    {
      "adversarial_sentence": "I will be traveling to Paris in May to attend a conference on sustainable development.",
      "explanation": "Ambiguous term 'May' used here as month and not as verb."
    },
    {
      "adversarial_sentence": "Wednesday is a talented musician who has been gaining popularity for her unique blend of jazz and electronic music.",
      "explanation": "Ambiguous term 'Wednesday' used here as person's name and not as day of the week."
    }
  ]
}


In [21]:
print(response['negative_sentences'])

{
    "adversarial_sentences": [
        {
            "sentence": "I saw her just the other day.",
            "explanation": "This sentence contains the term 'the other day', which is a non-specific time reference and should be avoided when labeling 'date'."
        },
        {
            "sentence": "She will arrive soon.",
            "explanation": "The term 'soon' is a non-specific time reference and should not be labeled as 'date' according to the guidelines."
        },
        {
            "sentence": "May I ask you a question?",
            "explanation": "The term 'May' in this sentence is ambiguous as it could refer to the month or be used as a verb, so it should not be labeled as 'date'."
        }
    ]
}


In [24]:
print(response['negative_sentences'])

{
  "adversarial_sentences": [
    {
      "sentence": "I met someone recently who told me they were born on a Wednesday.",
      "explanation": "This sentence contains the term 'recently' which is a non-specific time reference, violating the guideline to avoid labeling such terms."
    },
    {
      "sentence": "May I ask you to remind me about the meeting next week?",
      "explanation": "This sentence includes the term 'May' which is an ambiguous term that can refer to the month or be used as a verb, violating the guideline to exercise caution with such terms."
    },
    {
      "sentence": "I heard a rumor that soon we will have a new project starting.",
      "explanation": "This sentence contains the term 'soon' which is a non-specific time reference, violating the guideline to avoid labeling such terms."
    }
  ]
}


In [ ]:
print(response['negative_sentences'])

In [ ]:
print(response['negative_sentences'])

In [ ]:
print(response['negative_sentences'])

### get all definitions

In [31]:
negative_generated_examples = []
for ne in list(guidelines_per_ne_type.keys())[300:]:
    try:
        response = get_prompt_for_a_ne(ne, guidelines_per_ne_type, gpt_model, max_output_tokens=1000, n_sentences_to_generate=3)
        response['negative_sentences'] = json.loads(response['negative_sentences'])
        negative_generated_examples.append(response)
    except:
        with open(f"./adversarial_examples/{dataset_name}_err.json", 'w') as f:
            json.dump(negative_generated_examples, f, indent=2)
        print(f"Something went wrong while processing NE: {ne}")

with open(f"./adversarial_examples/{dataset_name}_300_423.json", 'w') as f:
    json.dump(negative_generated_examples, f, indent=2)

{'role': 'system', 'content': 'You are a helpful NER data annotator designed to output JSON.'}
{'role': 'user', 'content': 'You will be given a Named Entity and its associated annotation guidelines (delimited by triple quotes). \nCarefully read the Definition and Guidelines before answering to the instruction in the end. \n"""\nNamed Entity: \'role\', {"Definition": "\'role\' in the context of NER refers to a specific function, position, or responsibility within a system, organization, or context.", "Guidelines": "Do not label generic terms like \'worker\' or \'member\' unless they are part of a specific role (e.g., \'project manager\'). Be cautious with ambiguous terms that have multiple meanings in different contexts (e.g., \'driver\' as a job or a hardware component)."}\n"""\nInstruction: Leverage on the provided Definition and annotation Guidelines to generate 3 adversarial sentences containing negative occurrences of this Named Entity (negative occurrences are what the guidelines 

### END

In [60]:
with open('./adversarial_examples/ALL_pileNER_adv_examples_raw.json', 'r') as file:
    adversarial_examples_per_NE = json.load(file)

In [61]:
print(adversarial_examples_per_NE)
print(len(adversarial_examples_per_NE))

[{'named_entity': 'person', 'prompt_length': 232, 'output_length': 190, 'annotation_guidelines': {'Definition': "'person' signifies individuals, including real or fictional people, and refers to specific human beings.", 'Guidelines': "Avoid labeling generic terms like 'babies' or 'mothers' unless referring to specific individuals. Exercise caution with ambiguous references, such as 'Tongan Thor', which may include both a nickname and a real name."}, 'negative_sentences': {'adversarial_sentences': [{'sentence': 'The babies were crying loudly in the nursery, but the person in charge managed to calm them down.', 'explanation': "The term 'person' should not be used to refer to generic terms like 'babies'."}, {'sentence': 'Mothers play a crucial role in nurturing their children, ensuring they grow up to be responsible adults.', 'explanation': "The term 'person' should not be used to label generic terms like 'mothers'."}, {'sentence': 'Tongan Thor, the nickname of the famous rugby player, sc

In [62]:
missing_ne = set(ne_types_list) - set([sample['named_entity'] for sample in adversarial_examples_per_NE])
print(missing_ne)

set()


In [63]:
cleaned_adversarial_examples = {}
for sample in adversarial_examples_per_NE:
    named_entity = sample['named_entity']
    annotation_guidelines = sample['annotation_guidelines']
    negative_sentences = sample['negative_sentences']
    if isinstance(negative_sentences, dict):
        first_value = list(negative_sentences.values())[0]
        if isinstance(first_value, list):
            negative_sentences = first_value
    cleaned_adversarial_examples[named_entity] = {
        'named_entity': named_entity,
        'annotation_guidelines': annotation_guidelines,
        'negative_sentences': negative_sentences
    }


In [64]:
print(cleaned_adversarial_examples)

{'person': {'named_entity': 'person', 'annotation_guidelines': {'Definition': "'person' signifies individuals, including real or fictional people, and refers to specific human beings.", 'Guidelines': "Avoid labeling generic terms like 'babies' or 'mothers' unless referring to specific individuals. Exercise caution with ambiguous references, such as 'Tongan Thor', which may include both a nickname and a real name."}, 'negative_sentences': [{'sentence': 'The babies were crying loudly in the nursery, but the person in charge managed to calm them down.', 'explanation': "The term 'person' should not be used to refer to generic terms like 'babies'."}, {'sentence': 'Mothers play a crucial role in nurturing their children, ensuring they grow up to be responsible adults.', 'explanation': "The term 'person' should not be used to label generic terms like 'mothers'."}, {'sentence': 'Tongan Thor, the nickname of the famous rugby player, scored a try in the final minutes of the game, securing victor

In [65]:
print(len(cleaned_adversarial_examples))

423


In [66]:
with open(f"./adversarial_examples/ALL_pileNER_adv_examples.json", 'w') as f:
    json.dump(cleaned_adversarial_examples, f, indent=2)